In [1]:
!pip install requests pandas openpyxl schedule fpdf

In [2]:
import requests
import pandas as pd

def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": False,
        "price_change_percentage": "24h"
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        crypto_list = []
        for coin in data:
            crypto_list.append({
                "Name": coin["name"],
                "Symbol": coin["symbol"].upper(),
                "Current Price (USD)": coin["current_price"],
                "Market Cap": coin["market_cap"],
                "24h Volume": coin["total_volume"],
                "24h Change (%)": coin["price_change_percentage_24h"]
            })
        
        return pd.DataFrame(crypto_list)
    else:
        print("Error fetching data")
        return None

df = fetch_crypto_data()
df.head()  # Show the first few rows

,Name,Symbol,Current Price (USD),Market Cap,24h Volume,24h Change (%)
0,Bitcoin,BTC,97747.00,1938327628782,30117975064,0.41742
1,Ethereum,ETH,2687.82,323951673207,17657787204,1.66274
2,XRP,XRP,2.48,143144842641,4006914910,2.61527
3,Tether,USDT,1.00,141907649269,43915693712,0.00628
4,Solana,SOL,202.09,98618100291,4767625748,-0.56446


In [3]:
df.to_excel("crypto_data.xlsx", index=False)
print("Data saved to crypto_data.xlsx")

Data saved to crypto_data.xlsx


In [4]:
# Top 5 Cryptos by Market Cap
top_5 = df.nlargest(5, "Market Cap")

# Average Price
avg_price = df["Current Price (USD)"].mean()

# Highest & Lowest 24h Change
highest_change = df.nlargest(1, "24h Change (%)")
lowest_change = df.nsmallest(1, "24h Change (%)")

# Print analysis
print("\nTop 5 Cryptocurrencies by Market Cap:")
print(top_5[["Name", "Market Cap"]])

print(f"\nAverage Price of Top 50 Cryptos: ${avg_price:.2f}")

print("\nHighest 24h Change:")
print(highest_change[["Name", "24h Change (%)"]])

print("\nLowest 24h Change:")
print(lowest_change[["Name", "24h Change (%)"]])


Top 5 Cryptocurrencies by Market Cap:
       Name     Market Cap
0   Bitcoin  1938327628782
1  Ethereum   323951673207
2       XRP   143144842641
3    Tether   141907649269
4    Solana    98618100291

Average Price of Top 50 Cryptos: $4238.44

Highest 24h Change:
      Name  24h Change (%)
8  Cardano        15.55976

Lowest 24h Change:
      Name  24h Change (%)
30  MANTRA        -2.86326


In [5]:
from fpdf import FPDF

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", "B", 16)
pdf.cell(200, 10, "Cryptocurrency Market Analysis", ln=True, align="C")

pdf.ln(10)
pdf.set_font("Arial", "", 12)
pdf.multi_cell(0, 10, "This report presents an analysis of the top 50 cryptocurrencies, "
                       "focusing on market capitalization, price trends, and price fluctuations over 24 hours.")

pdf.ln(5)
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Key Findings:", ln=True)

pdf.set_font("Arial", "", 12)
pdf.cell(0, 10, f"- The average price of the top 50 cryptocurrencies is ${avg_price:.2f}.", ln=True)
pdf.cell(0, 10, f"- The highest 24-hour change is {highest_change.iloc[0]['Name']} ({highest_change.iloc[0]['24h Change (%)']:.2f}%).", ln=True)
pdf.cell(0, 10, f"- The lowest 24-hour change is {lowest_change.iloc[0]['Name']} ({lowest_change.iloc[0]['24h Change (%)']:.2f}%).", ln=True)

pdf.ln(5)
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Top 5 Cryptocurrencies by Market Cap:", ln=True)

pdf.set_font("Arial", "", 12)
for i, row in top_5.iterrows():
    pdf.cell(0, 10, f"- {row['Name']} (Market Cap: ${row['Market Cap']:,})", ln=True)

pdf.ln(5)
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Conclusion:", ln=True)
pdf.set_font("Arial", "", 12)
pdf.multi_cell(0, 10, "This analysis provides insights into the top cryptocurrencies and their price changes. "
                       "The data can help investors understand trends and make informed decisions.")

pdf.output("crypto_analysis.pdf")
print("PDF report generated successfully.")

PDF report generated successfully.


In [6]:
import pandas as pd

try:
    df = pd.read_excel("crypto_data.xlsx", engine="openpyxl")
    print("File loaded successfully!")
    print(df.head())  # Show first few rows
except Exception as e:
    print("Error:", e)

File loaded successfully!
       Name Symbol  Current Price (USD)     Market Cap   24h Volume  \
0   Bitcoin    BTC             97747.00  1938327628782  30117975064   
1  Ethereum    ETH              2687.82   323951673207  17657787204   
2       XRP    XRP                 2.48   143144842641   4006914910   
3    Tether   USDT                 1.00   141907649269  43915693712   
4    Solana    SOL               202.09    98618100291   4767625748   

   24h Change (%)  
0         0.41742  
1         1.66274  
2         2.61527  
3         0.00628  
4        -0.56446  
